# Out-of-scope Doc2Vec
Creates Doc2Vec representations from the OOS list.

In [1]:
# Imports
import re
from collections import Counter
from pprint import pprint

import pandas as pd
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE


Generate Doc2Vec for websites.

In [3]:
data_path = "../uri_scores.csv"
file = pd.read_csv(data_path)

def iter_urls(file):
    for i, row in file.iterrows():
        url, text = row["original_url"], str(row["text"])
        yield url, text


class TaggedWebpageDocument(object):
    def __iter__(self):
        for url, text in iter_urls(file):
            words = [c for c in re.split(r"\s+", re.sub(r"[^\w\s]+", " ", text)) if len(c) > 0]
            yield TaggedDocument(words, [url])


documents = TaggedWebpageDocument()
model = Doc2Vec(documents, window=7, vector_size=256, workers=4)
model.save("oos-doc2vec")

/home/rolv-arild/PycharmProjects/soc/venv/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
model = Doc2Vec.load("oos-doc2vec")

Small tests.

In [4]:
pprint(model.docvecs.most_similar(positive=["https://itunes.apple.com"], topn=10))
pprint(model.docvecs.most_similar(positive=["https://begrep.difi.no"], topn=10))
pprint(model.docvecs.most_similar(positive=["https://www.sharp.fi"], topn=10))

[('http://ax.itunes.apple.com', 0.9562076330184937),
 ('http://itunes.apple.com', 0.9437013864517212),
 ('https://geo.itunes.apple.com', 0.9404727220535278),
 ('http://store.apple.com', 0.8602949380874634),
 ('http://livepage.apple.com', 0.840777575969696),
 ('http://apple.com', 0.8331955671310425),
 ('http://www.apple.com', 0.8172075152397156),
 ('https://www.apple.com', 0.8163220286369324),
 ('https://support.apple.com', 0.8123588562011719),
 ('http://www.etudiemploi.fr', 0.8099346160888672)]
[('http://blogg.jaktogdvd.no', 0.7379841208457947),
 ('http://www.alleteller.no', 0.7270657420158386),
 ('https://www.aid.no', 0.7266299724578857),
 ('http://www.norgesstigen.no', 0.7253384590148926),
 ('http://fsweb.no', 0.7209904193878174),
 ('https://www.anbuddirekte.no', 0.7139006853103638),
 ('http://algarheim.backe.no', 0.7104812264442444),
 ('http://reklameregler.lmi.no', 0.7098019123077393),
 ('http://www.mednytt.no', 0.709213376045227),
 ('http://www.presse.no', 0.7085011005401611)]
[('

Run dimensionality reduction.

In [ ]:
pca = PCA()

trans = pca.fit_transform(model.docvecs.vectors_docs)

print(trans.shape)

# Plot explained variance (information kept) vs number of components
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

In [ ]:
reduced = TSNE().fit_transform(trans)

np.save("veidemann-reduced.npy", reduced)

In [ ]:
reduced = np.load("veidemann-reduced.npy")


Write to CSV for later analysis.

In [5]:
print(len(file), print(model.docvecs.vectors_docs.shape))
vector_df = pd.DataFrame(data=model.docvecs.vectors_docs, columns=[f"v{i}" for i in range(256)])
conc = pd.concat([file, vector_df], axis=1)
conc.to_csv("oos-doc2vec.csv", index=False)

(69858, 256)
69858 None


In [6]:
# Find low scoring websites that are most similar to high scoring websites.
pprint(model.docvecs.most_similar(
    positive=[row["original_url"] for i, row in conc.iterrows() if row["norvegica_score"] > 0.5], topn=30))

[('https://ec.europa.eu', 0.9650459289550781),
 ('http://cassandrarhodin.com', 0.9630915522575378),
 ('http://www.norwegianicons.no', 0.9577521085739136),
 ('http://www.everycar.ph', 0.9563934803009033),
 ('https://www.scanjet.se', 0.9559930562973022),
 ('http://www.cimtshow.com', 0.9548647999763489),
 ('http://www.everycar.com.tw', 0.9545629024505615),
 ('http://www.dinosplattsburgh.com', 0.9534910917282104),
 ('https://minside.sb1kapital.no', 0.9534136056900024),
 ('http://www.annelisenorheim.no', 0.9523912668228149),
 ('http://www.antonsoggiu.com', 0.9522300958633423),
 ('http://www.cieme.org.cn', 0.9517980217933655),
 ('http://byggeplass.com', 0.9517480134963989),
 ('http://www.explorauto.com.bo', 0.9497545957565308),
 ('http://www.tried.com.cn', 0.9488672614097595),
 ('http://www.hallingkart.no', 0.9476903080940247),
 ('https://radio.disney.com', 0.9475408792495728),
 ('http://www.utmark.biz', 0.9474514126777649),
 ('http://www.krasznahorkai.hu', 0.9473844766616821),
 ('http://www

For converting to [TensorFlow embedding projector](https://projector.tensorflow.org/) compatible format

In [2]:
fn = "oos-doc2vec.csv"
csv = pd.read_csv(fn, index_col=False)

csv = csv.dropna(subset=[f"v{i}" for i in range(256)])
csv = csv.fillna("missing")

filt = "v\d+"
vec = csv.filter(regex=filt)
meta = csv.drop(vec.columns, axis=1)
# meta = meta.drop("text", axis=1)
for column in meta:
    try:
        meta[column] = meta[column].str.replace("\n", " ")
    except AttributeError:
        pass

meta = meta.set_index("original_url")

vec.to_csv(fn.replace(".csv", "-vec.tsv"), header=False, sep="\t", index=False)
meta.to_csv(fn.replace(".csv", "-meta.tsv"), sep="\t", index=True)

For Weka compatibility

In [8]:
import arff
conc_notxt = conc.drop("text", axis=1).fillna("missing")
arff.dump("oos.arff", conc_notxt.values, "ooslist", conc_notxt.columns)

Generate bar charts and histograms for DataFrame

In [27]:
nominal = (1,2,3,4,6,7,10,11,14,15,23)

for index in range(len(meta.columns)):
    column = meta.iloc[:, index]
    if index+1 not in nominal:
        if column.dtype == np.float64:
            chart = plt.hist(column.values, bins=101, bottom=0)
        elif column.dtype == np.int64:
            chart = plt.hist(column.values, bins=min(max(column.values)+1, 101), bottom=0)
        else:
            continue
    else: 
        counter = Counter(column)
        labels, values = zip(*counter.most_common(20))
        indexes = np.arange(len(labels))
        width=1
        chart = plt.bar(indexes, values, width)
        plt.xticks(indexes, labels, rotation=90)
    plt.semilogy()
    plt.suptitle(column.name, y=0.95)
    # plt.tight_layout()
    plt.savefig(f"{column.name}.png", bbox_inches="tight")
    plt.clf()

<Figure size 432x288 with 0 Axes>